In [95]:
import pandas as pd
import collections
from etl_gestores import etl_nota

In [ ]:
df_gestao = etl_nota()

In [ ]:
df_gestao.head(3)

In [4]:
gestores = {}
gestores['id_gestor'] = []
gestores['n_atuacoes'] = []
gestores['nota_var'] = []
for nome, dados in df_gestao.groupby('id_gestor'): 
    gestores['id_gestor'].append(nome)
    gestores['n_atuacoes'].append(dados.n_atuacoes.iloc[0])
    gestores['nota_var'].append(dados.nota_var.iloc[0])   

gestores = pd.DataFrame(data=gestores)
#gestores = gestores[gestores.n_atuacoes >= 2]

In [5]:
gestores.head(3)

,id_gestor,n_atuacoes,nota_var
0,131346,11,0.636364
1,131376,7,-0.428571
2,131393,6,8.333333


In [6]:
gestores['primeira_viagem'] = gestores.n_atuacoes.apply(lambda x: 0 if x > 1 else 1)

In [7]:
gestores.loc[:,['nota_var','primeira_viagem']].corr()  # primeira viagem e ruim

,nota_var,primeira_viagem
nota_var,1.00000,-0.02266
primeira_viagem,-0.02266,1.00000


# GESTOR QUE FALTA DEVE SER RUIM

In [8]:
arquivo = 'D:\\DESAFIO_SEDUC\\dados\\ausncias-por-servidor-All-2020-04-13_1011\\BASE_AUSENCIAS_1118.csv'
ausen1118 = pd.read_csv(arquivo, sep=';')
arquivo = 'D:\\DESAFIO_SEDUC\\dados\\ausncias-por-servidor-All-2020-04-13_1011\\BASE_AUSENCIAS_0419.csv'
ausen0419 = pd.read_csv(arquivo, sep=';')
arquivo = 'D:\\DESAFIO_SEDUC\\dados\\ausncias-por-servidor-All-2020-04-13_1011\\BASE_AUSENCIAS_1119.csv'
ausen1119 = pd.read_csv(arquivo, sep=';')
ausen1118.columns = ausen1119.columns
ausen0419.columns = ausen1119.columns
ausen = ausen1118.append(ausen0419).append(ausen1119)

In [9]:
CATEGORIAS = ['TT_DIAS_FALTA_MEDICA', 'TT_DIAS_FALTA_JUST', 'TT_DIAS_FALTA_INJUST',
              'TT_DIAS_FALTA_ABONADA', 'TT_DIAS_LIC_PREMIO', 'TT_DIAS_LIC_GESTANTE', 
              'TT_DIAS_LIC_ACID_TRAB', 'TT_DIAS_LIC_INTER_PARTIC']

aus = {}
aus['id_gestor'] = []
aus['dias'] = []
for _, row in ausen.iterrows():
    try:
        id_interno = int(row.loc['id_interno'])
    except:
        continue
        
    dias = 0
    for cat in CATEGORIAS:
        try:
            dia_x = int(row.loc[cat])
        except:
            dia_x = 0
        dias += dia_x
    aus['id_gestor'].append(id_interno)
    aus['dias'].append(dias)

aus = pd.DataFrame(aus)

aus2 = {}
aus2['id_gestor'] = []
aus2['teve_falta'] = []
teve_falta = lambda x: 1 if x > 0 else 0
for nome, dados in aus.groupby('id_gestor'):
    aus2['id_gestor'].append(nome)
    aus2['teve_falta'].append(teve_falta(dados.dias.sum()))
    
aus = pd.DataFrame(aus2)    

In [10]:
gest_falta = pd.merge(gestores, aus, on='id_gestor',how='left')
gest_falta.teve_falta = gest_falta.teve_falta.fillna(0)

In [11]:
gest_falta.loc[:,['nota_var','teve_falta']].corr()  # correlacao para mostrar que influencia

,nota_var,teve_falta
nota_var,1.000000,-0.013066
teve_falta,-0.013066,1.000000


# VER A FORMACAO DO GESTOR

In [12]:
arquivo = 'D:\\DESAFIO_SEDUC\\dados\\formao-por-servidor-All-2020-04-13_1013\\BASE_FORMACAO_0419a.csv'
formac0419 = pd.read_csv(arquivo, sep=';', encoding ='latin-1')
arquivo = 'D:\\DESAFIO_SEDUC\\dados\\formao-por-servidor-All-2020-04-13_1013\\BASE_FORMACAO_1118.csv'
formac1118 = pd.read_csv(arquivo, sep=';', encoding ='latin-1')
arquivo = 'D:\\DESAFIO_SEDUC\\dados\\formao-por-servidor-All-2020-04-13_1013\\BASE_FORMACAO_1119a.csv'
formac1119a = pd.read_csv(arquivo, sep=';', encoding ='latin-1')
formac = formac0419.append(formac1118).append(formac1119a)

C:\Users\admprime\Miniconda3\envs\fklearn\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [13]:
def pontua_formacao(formacao):
    # graduacao/especializacao + 1
    # doutorado +2
    
    if formacao == 'LICENCIATURA':
        return 1
    elif formacao == 'LICENCIATURA + BACHARELADO/TECNÌÒLOGO + ESPECIALIZAÌàÌÄO':
        return 3
    elif formacao == 'LICENCIATURA + ESPECIALIZAÌàÌÄO':
        return 2
    elif formacao == 'ENSINO MÌäDIO':
        return 0
    elif formacao == 'LICENCIATURA + BACHARELADO/TECNÌÒLOGO':
        return 2
    elif formacao == 'BACHARELADO/TECNÌÒLOGO':
        return 1
    elif formacao == 'BACHARELADO/TECNÌÒLOGO':
        return 1
    elif formacao == 'S/INFO':
        return 1
    elif formacao == 'LICENCIATURA + ESPECIALIZAÌàÌÄO + MESTRADO':
        return 3
    elif formacao == 'LICENCIATURA + MESTRADO':
        return 2
    elif formacao == 'LICENCIATURA + BACHARELADO/TECNÌÒLOGO + MESTRADO':
        return 3
    elif formacao == 'ENSINO FUNDAMENTAL':
        return 0
    elif formacao == 'LICENCIATURA + BACHARELADO/TECNÌÒLOGO + ESPECIALIZAÌàÌÄO + MESTRADO':
        return 4
    elif formacao == 'LICENCIATURA + MESTRADO+DOUTORADO':
        return 4
    elif formacao == 'LICENCIATURA + BACHARELADO/TECNÌÒLOGO + MESTRADO + DOUTORADO':
        return 5
    elif formacao == 'APERF/EXTENSÌÄO CULTURAL':
        return 0
    elif formacao == 'LICENCIATURA + ESPECIALIZAÌàÌÄO + MESTRADO + DOUTORADO':
        return 5
    elif formacao == 'BACHARELADO/TECNÌÒLOGO + ESPECIALIZAÌàÌÄO':
        return 2
    elif formacao == 'LICENCIATURA + DOUTORADO':
        return 3
    elif formacao == 'LICENCIATURA + ESPECIALIZAÌàÌÄO+DOUTORADO':
        return 4
    elif formacao == 'BACHARELADO/TECNÌÒLOGO + MESTRADO':
        return 2
    elif formacao == 'LICENCIATURA + BACHARELADO/TECNÌÒLOGO+ESPECIALIZAÌàÌÄO+DOUTORADO':
        return 5
    elif formacao == 'ESPECIALIZAÌàÌÄO':
        return 2
    elif formacao == 'BACHARELADO/TECNÌÒLOGO + MESTRADO + DOUTORADO':
        return 4
    elif formacao == 'LICENCIATURA + BACHARELADO/TECNÌÒLOGO + DOUTORADO':
        return 4
    elif formacao == 'LICENCIATURA + BACHARELADO/TECNÌÒLOGO+ESPECIALIZAÌàÌÄO + MESTRADO + DOUTORADO':
        return 6
    elif formacao == 'ESPECIALIZAÌàÌÄO+MESTRADO':
        return 3
    elif formacao == 'BACHARELADO/TECNÌÒLOGO + ESPECIALIZAÌàÌÄO + MESTRADO':
        return 3
    elif formacao == 'BACHARELADO/TECNÌÒLOGO + DOUTORADO':
        return 3
    elif formacao == 'MESTRADO':
        return 2
    else:
        return 1


In [14]:
formac['formacao_pontos'] = formac.FORMACAO.apply(pontua_formacao)
formac = formac.drop(formac[formac.id_interno.isnull()].index)
formac.id_interno = formac.id_interno.astype(int)
formac = formac.rename({'id_interno': 'id_gestor'}, axis=1)
formac = formac.loc[:,['id_gestor','formacao_pontos']]

In [15]:
formac2 = {}
formac2['id_gestor'] = []
formac2['formacao_pontos'] = []
for nome, dados in formac.groupby('id_gestor'):
    formac2['id_gestor'].append(nome)
    formac2['formacao_pontos'].append(dados.formacao_pontos.max())
    
formac = pd.DataFrame(formac2)

In [16]:
gest = pd.merge(gest_falta, formac, on='id_gestor',how='left')
gest.formacao_pontos = gest.formacao_pontos.fillna(1)

In [17]:
gest.loc[:,['nota_var','formacao_pontos']].corr()

,nota_var,formacao_pontos
nota_var,1.000000,-0.022544
formacao_pontos,-0.022544,1.000000


# VER CARACTERISTICAS DO SERVIDOR

In [38]:
arquivo = 'dados\servidores-ativos-por-unidade-All-2020-04-13_1012\BASE_SERVIDORES_ATIVOS_0419.csv'
servidor0419 = pd.read_csv(arquivo, sep=';')
arquivo = 'dados\servidores-ativos-por-unidade-All-2020-04-13_1012\BASE_SERVIDORES_ATIVOS_1118.csv'
servidor1118 = pd.read_csv(arquivo, sep=';')
arquivo = 'dados\servidores-ativos-por-unidade-All-2020-04-13_1012\BASE_SERVIDORES_ATIVOS_1119.csv'
servidor1119 = pd.read_csv(arquivo, sep=';')
servidor = servidor0419.append(servidor1118).append(servidor1119)
servidor = servidor.drop(servidor[servidor.id_interno.isnull()].index)

C:\Users\admprime\Miniconda3\envs\fklearn\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (11,36) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [44]:
servidor.id_interno = servidor.id_interno.astype(int)
servidor['SEXO_MASC'] = servidor.SEXO.apply(lambda x: 1 if x == 'M' else 0)

In [51]:
serv = {}
serv['id_gestor'] = []
serv['idade'] = []
serv['anos_trabalho'] = []
serv['sexo'] = []
for nome, dados in servidor.groupby('id_interno'):
    serv['id_gestor'].append(nome)
    serv['idade'].append(dados.IDADE.max())
    serv['anos_trabalho'].append(dados.ANOS_TRAB_CARGO_C.max())
    serv['sexo'].append(dados.SEXO_MASC.max())
serv = pd.DataFrame(serv)

In [78]:
gest_serv = pd.merge(gest, serv, on='id_gestor',how='left')

In [79]:
gest_serv.loc[:,['nota_var','idade','anos_trabalho','sexo']].corr()

,nota_var,idade,anos_trabalho,sexo
nota_var,1.000000,-0.007257,-0.017737,0.022639
idade,-0.007257,1.000000,0.432547,-0.090545
anos_trabalho,-0.017737,0.432547,1.000000,-0.097796
sexo,0.022639,-0.090545,-0.097796,1.000000


### ESTATISTICA DESCRIVA

In [85]:
gest_serv = gest_serv.drop(gest_serv[gest_serv.idade.isnull()].index)

In [86]:
gest_serv.idade.describe(percentiles=[0.1,0.9])

count    27769.000000
mean        47.585797
std          8.135821
min         25.000000
10%         37.000000
50%         48.000000
90%         58.000000
max         73.000000
Name: idade, dtype: float64

In [88]:
gest_serv.anos_trabalho.describe(percentiles=[0.25,0.75])

count    27769.000000
mean        12.797292
std          6.359834
min          0.000000
25%          8.000000
50%         13.000000
75%         17.000000
max         46.000000
Name: anos_trabalho, dtype: float64

In [90]:
gest_serv[gest_serv.anos_trabalho <= 8].nota_var.mean()

6.621039024565032

In [91]:
gest_serv[(gest_serv.anos_trabalho > 8) &
         (gest_serv.anos_trabalho <= 13)].nota_var.mean()

6.4172509126018475

In [92]:
gest_serv[(gest_serv.anos_trabalho > 13) &
         (gest_serv.anos_trabalho <= 17)].nota_var.mean()

6.484498037456371

In [93]:
gest_serv[gest_serv.anos_trabalho > 17].nota_var.mean()

6.251948018980994

In [68]:
gest_serv.sexo = gest_serv.sexo.fillna(1)
gest_serv.idade = gest_serv.idade.fillna(gest_serv.idade.median())
gest_serv.anos_trabalho = gest_serv.anos_trabalho.fillna(gest_serv.anos_trabalho.median())

In [69]:
gest_serv.loc[:,['nota_var','idade','anos_trabalho','sexo']].corr()

,nota_var,idade,anos_trabalho,sexo
nota_var,1.000000,-0.004913,-0.013196,0.027114
idade,-0.004913,1.000000,0.432864,-0.037471
anos_trabalho,-0.013196,0.432864,1.000000,-0.051018
sexo,0.027114,-0.037471,-0.051018,1.000000


In [77]:
gest_serv.to_csv('avaliacao_gestores.csv', index=False, sep=';')